## Evaluate transfer learning on Turnedtable Watertank (Dataset 2)  using an SVM Classifier: Self-supervised approach 

In [1]:
import tensorflow as tf
import h5py
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import sys
import os
sys.path.append('../../../')
sys.path.append('../../')

### Configure Tensorflow for GPU device

In [2]:
tf.config.experimental_run_functions_eagerly(True)
print("[INFO] Tensorflow Version:", tf.__version__)

if tf.config.list_physical_devices("GPU") and tf.test.is_built_with_cuda():
    print("[INFO] Tensorflow built with CUDA")
    print("[INFO] Number GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    print("[INFO] List of GPU devices:", tf.config.list_physical_devices("GPU"))
    physical_devices = tf.config.list_physical_devices("GPU")
    # tf.config.experimental.set_memory_growth(physical_devices[0], True)
    for gpu in physical_devices:
        tf.config.experimental.set_memory_growth(gpu, True)

else:
    print("[ERROR] GPU not detected, make sure tensorflow-gpu is installed and that GPU is recognized")
    exit()

[INFO] Tensorflow Version: 2.2.0
[INFO] Tensorflow built with CUDA
[INFO] Number GPUs Available:  1
[INFO] List of GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Define utilities

In [3]:
# utilities
def flatten(x):
    return x.reshape((x.shape[0], -1))

def classSampling(X, y, samplesPerClass, numberOfClasses):
    X_ret = np.zeros((samplesPerClass * numberOfClasses, X.shape[1]), dtype = np.float32)
    y_ret = np.zeros((samplesPerClass * numberOfClasses), dtype = np.uint8)
    count = 0

    for classIdx in range(numberOfClasses):
        indices = np.where(y == classIdx)[0]

        #if len(indices) < samplesPerClass:
        #    raise IndexError("Not enough samples for class {} to produce {} samples per class. Only {} class samples available".format(classIdx, samplesPerClass, len(indices)))

        doResample = len(indices) < samplesPerClass

        chosenIndices = np.random.choice(indices, samplesPerClass, replace = doResample)

        for ci in chosenIndices:
            X_ret[count] = X[ci]
            y_ret[count] = y[ci]

            count += 1

    return X_ret, y_ret

### Define dataset and dataloader

In [4]:
class SonarTurnedTableSupervised(object):
    def __init__(self, file_path):
        self.file_path = file_path

    def _normalize_images(self, images):
        """
        Normalize sonar images by 1/255.
        """
        return [element/255.0 for element in images]

    def get_sonar_data(self):
        """
        Reads from HDF5 file containing sonar data (resized to fix dims).
        Returns list of np arrays containing image data.
        """

        print("[INFO] Retrieving Sonar Turned Table Supervised Data")

        with h5py.File(self.file_path, "r") as f:
            # list all groups
            print("hdf5 dataset keys: %s" % f.keys())

            # get images and labels
            x_train = f["x_train"][...].astype(np.float32)
            y_train = f["y_train"][...]

            x_test = f["x_test"][...].astype(np.float32)
            y_test = f["y_test"][...]

            _, x_val, _, y_val = train_test_split(x_test, y_test, train_size=0.5)

            print("[INFO] Data dimensions")
            print("Train", len(x_train))
            print("Val", len(x_val))
            print("Test", len(x_test))

            # matias normalization
            # multiply by 255 because hdf5 file comes as 1/255
            x_train *= 255.0
            x_val *= 255.0
            x_test *= 255.0

            x_train -= 84.51
            x_val -= 84.51
            x_test  -= 84.51

        return (x_train, y_train), (x_val, y_val), (x_test, y_test)
    
def load_sonar_turnedtable_supervised(file_path):
    """
    Loads test data from turnedtable dataset.
    """
    print()
    print("[INFO] Loading Tenorflow dataset")

    dataset_object = SonarTurnedTableSupervised(file_path)

    # Read data
    (x_train, y_train), (x_val, y_val), (x_test, y_test) = dataset_object.get_sonar_data()

    # Train data
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
    train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(len(x_train))
    train_dataset = train_dataset.prefetch(25)

    # Validation data
    # val_dataset = tf.data.Dataset.from_tensor_slices((x_val, labels_val))
    # val_dataset = val_dataset.shuffle(buffer_size=len(x_val)).batch(batch_size)
    # val_dataset = val_dataset.prefetch(25)

    # Test data
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_dataset = test_dataset.shuffle(buffer_size=len(x_test)).batch(len(x_test)) # feed full test set
    test_dataset = test_dataset.prefetch(25)

    print()
    print("[INFO] Tensorflow data dimensions")
    # print(train_dataset)
    # print(val_dataset)
    print(test_dataset)

    # return train_dataset, val_dataset, test_dataset
    return train_dataset, test_dataset

###  Load Pretrained Model

In [5]:
from architectures.squeezenet import squeezenet

PRETRAINED_NUM_CLASSES = 4 # 11 supervised, 4 self-supervised
input_shape = [96, 96, 1]

model_name = "squeezenet"
pretraining_mode = "self_supervised_learning"
# layers = ["batch_norm_48", "batch_norm_49", "batch_norm_50"] # TODO: check layer names
layers = ["batch_normalization_8", "batch_normalization_9", "global_average_pooling2d"]
model = squeezenet(input_shape, PRETRAINED_NUM_CLASSES)

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 96, 96, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 48, 48, 8)    400         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 48, 48, 8)    32          conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 24, 24, 8)    0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [6]:
# check weights BEFORE loading pretrained model (second conv layer)
print(model.layers[1].get_weights()[0])

[[[[-0.00544592 -0.01632494  0.01687813  0.04675952  0.01727858
     0.08048141  0.04251303 -0.1139203 ]]

  [[-0.09565575  0.10050267 -0.09226272 -0.11019079  0.02015136
    -0.09789215 -0.02381014 -0.06458307]]

  [[ 0.09331498 -0.01439317  0.01874527 -0.0148453  -0.07260352
    -0.05460784  0.00416581  0.0907032 ]]

  [[ 0.09892149 -0.04754139 -0.0133201   0.08658461  0.02463846
    -0.06913388  0.09681627  0.02571903]]

  [[ 0.03571258 -0.05388857 -0.05642048  0.00077745 -0.00709493
     0.10368766 -0.10818358  0.02545564]]

  [[-0.0533876  -0.06222044  0.01957692 -0.02066834  0.11260329
    -0.02103198 -0.06476174  0.11315957]]

  [[-0.03850563 -0.09236903 -0.05340952  0.04758163 -0.01292292
     0.02692783 -0.04538411  0.09803072]]]


 [[[-0.05424626 -0.04718952  0.07769749  0.01299366 -0.08162719
    -0.11072285 -0.08204272  0.07492848]]

  [[ 0.02731937  0.03558099  0.02277398  0.04992607 -0.08838123
     0.08094735 -0.11646322 -0.07953104]]

  [[-0.03841574 -0.03013471 -0.1091

In [7]:
# path to pretrained model checkpoint
pretrained_checkpoint_prefix = os.path.join("../../../pretraining/results/" + pretraining_mode + "/checkpoints/sonar1/" + model_name + "/batch_size_128/96x96_substract_mean_online_aug_width_32")

# optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.5, epsilon=1)

# define pretrained checkpoint model
checkpoint_pretrained = tf.train.Checkpoint(step=tf.Variable(1), optimizer=optimizer, model=model) # 4 ssl, 11 sl
manager_pretrained = tf.train.CheckpointManager(checkpoint_pretrained, pretrained_checkpoint_prefix, max_to_keep=3)

# restore model weights
checkpoint_pretrained.restore(manager_pretrained.latest_checkpoint)

if manager_pretrained.latest_checkpoint:
    print("[INFO] Pretrained checkpoint restored correctly: {}".format(manager_pretrained.latest_checkpoint))
else:
    print("[INFO] Could not restore pretrained checkpoint correctly, make sure path to pre-trained folder is correct.")

[INFO] Pretrained checkpoint restored correctly: ../../../pretraining/results/self_supervised_learning/checkpoints/sonar1/squeezenet/batch_size_128/96x96_substract_mean_online_aug_width_32/ckpt-21


In [8]:
# check weights AFTER loading pretrained model (second conv layer)
print(model.layers[1].get_weights()[0])

[[[[-0.06216595 -0.03900114 -0.02578199 -0.02512501 -0.00926176
    -0.0772492   0.08437654  0.09840393]]

  [[ 0.08783991 -0.10164145 -0.02722576 -0.08092385 -0.0152075
    -0.09763503  0.01076004  0.02054841]]

  [[-0.03476039 -0.01866436 -0.11986611  0.05831946 -0.11343325
    -0.02424521 -0.06752407  0.00966931]]

  [[-0.09956822 -0.04149697 -0.0852205   0.10161399 -0.02028412
    -0.09849622 -0.01819678 -0.09657682]]

  [[ 0.05221378  0.01381214  0.00276488  0.03069827  0.05144576
    -0.00697406  0.05501994 -0.01501517]]

  [[ 0.04223234  0.10458919 -0.04650391  0.00161301  0.02099225
     0.10271543  0.11928242 -0.08690233]]

  [[ 0.08221544 -0.0049354  -0.05628303  0.06656159  0.09399419
     0.07261222 -0.02798264 -0.08347652]]]


 [[[ 0.00047691  0.07692079  0.01617629  0.07047198  0.08686607
    -0.0433566   0.00916725  0.04945499]]

  [[-0.02974954  0.01272425  0.0555886   0.09433492 -0.01387227
     0.04432397  0.13563244  0.00651278]]

  [[-0.09382562  0.05086119  0.03260

### Define models up to intermediate layers


In [9]:
print("Intermediate layers from model:", layers)

Intermediate layers from model: ['batch_normalization_8', 'batch_normalization_9', 'global_average_pooling2d']


In [10]:
# https://stackoverflow.com/questions/63297838/how-can-i-obtain-the-output-of-an-intermediate-layer-feature-extraction
batch_normalization_8 = tf.keras.Model(inputs=model.get_layer("input_1").output, 
                                     outputs=model.get_layer(layers[0]).output)

batch_normalization_9 = tf.keras.Model(inputs=model.get_layer("input_1").output, 
                                     outputs=model.get_layer(layers[1]).output)

global_average_pooling2d = tf.keras.Model(inputs=model.get_layer("input_1").output, 
                                     outputs=model.get_layer(layers[2]).output)


### Generate vector embeddings for train and test data (up to n-th layer)

In [11]:
# define tensorflow dataset
data_dir = "../../../../../../datasets/sonar_turntable_dataset_2/marine-debris-turntable-classification-object_classes-platform-96x96.hdf5"
train_dataset, test_dataset = load_sonar_turnedtable_supervised(data_dir)

# load tensorflow tensors individually
x_train, y_train = next(iter(train_dataset))
x_test, y_test = next(iter(test_dataset))


[INFO] Loading Tenorflow dataset
[INFO] Retrieving Sonar Turned Table Supervised Data
hdf5 dataset keys: <KeysViewHDF5 ['class_names', 'x_test', 'x_train', 'y_test', 'y_train']>
[INFO] Data dimensions
Train 1505
Val 323
Test 645

[INFO] Tensorflow data dimensions
<PrefetchDataset shapes: ((None, 96, 96, 1), (None,)), types: (tf.float32, tf.int64)>


In [13]:
# perform a forward pass to generate embeddings (both train and test data) (for each n-th layer)

# NOTE: when doing this all tensors are loaded into memory, this saturates NVIDIA memory (nvidia-smi)
x_train_batch_normalization_8 = batch_normalization_8([x_train], training=False)
x_train_batch_normalization_9 = batch_normalization_9([x_train], training=False)
x_train_global_average_pooling2d = global_average_pooling2d([x_train], training=False)

x_test_batch_normalization_8 = batch_normalization_8([x_test], training=False)
x_test_batch_normalization_9 = batch_normalization_9([x_test], training=False)
x_test_global_average_pooling2d = global_average_pooling2d([x_test], training=False)

ResourceExhaustedError: OOM when allocating tensor with shape[1505,24,24,64] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Conv2D]

In [15]:
x_train_batch_norm_48

<tf.Tensor: shape=(1505, 6, 6, 512), dtype=float32, numpy=
array([[[[1.21386254e+00, 4.89381775e-02, 8.45410645e-01, ...,
          0.00000000e+00, 4.96995270e-01, 9.05933157e-02],
         [1.40980411e+00, 1.15261090e+00, 8.35694194e-01, ...,
          0.00000000e+00, 5.93474329e-01, 4.82767463e-01],
         [1.82574883e-01, 6.56372130e-01, 5.60133994e-01, ...,
          0.00000000e+00, 6.67990267e-01, 9.42384839e-01],
         [1.29431516e-01, 1.25769377e+00, 2.52908558e-01, ...,
          0.00000000e+00, 5.47911167e-01, 1.50075305e+00],
         [0.00000000e+00, 7.05164850e-01, 0.00000000e+00, ...,
          0.00000000e+00, 3.08221430e-01, 7.03265250e-01],
         [1.43316180e-01, 8.97599578e-01, 6.38682246e-01, ...,
          0.00000000e+00, 0.00000000e+00, 6.05568409e-01]],

        [[1.58779538e+00, 2.70380199e-01, 6.62188411e-01, ...,
          0.00000000e+00, 9.18517351e-01, 0.00000000e+00],
         [1.36984611e+00, 1.52387643e+00, 6.65007174e-01, ...,
          5.64474225e-

### Transfer Learning setup: classification with subsamples per object class (few shot learning)

In [16]:
# transfer learning params
# SAMPLES_PER_CLASS = [1, 5, 10, 20, 30, 40, 50] # NOTE: taking more samples per class since it is 88 for 50
# SAMPLES_PER_CLASS = [10, 20, 30, 40, 50, 70, 90, 110, 130, 150, len(x_test)]
SAMPLES_PER_CLASS = [10, 20, 30, 40, 50, 80, 110, 140, 170, 200]
TRIALS = 10

NUM_CLASSES_WATERTANK = 11
NUM_CLASSES_TURNEDTABLE = 12

In [17]:
# Flatten train & test data for SVM
x_train_batch_norm_48 = flatten(x_train_batch_norm_48.numpy())
x_train_batch_norm_49 = flatten(x_train_batch_norm_49.numpy())
x_train_batch_norm_50 = flatten(x_train_batch_norm_50.numpy())

x_test_batch_norm_48 = flatten(x_test_batch_norm_48.numpy())
x_test_batch_norm_49 = flatten(x_test_batch_norm_49.numpy())
x_test_batch_norm_50 = flatten(x_test_batch_norm_50.numpy())

In [18]:
# these two are not modified (only x_test, x_train)
print(y_train.numpy().shape)
print(y_test.numpy().shape)

y_train = y_train.numpy() # convert from tf tensor --> numpy
y_test = y_test.numpy()

(1505,)
(645,)


### Run svm tl evaluation with spc for each n-th layer

In [19]:
def train_svm_with_spc(x_train, y_train, x_test, y_test):
    """
    Takes embeddings from pretrained model and evaluates transfer learning 
    with few samples per class.
    """
    # NOTE: svm takes original labels (not one-hot encoding)
    for spc in SAMPLES_PER_CLASS:
        accuracies = []

        for i in range(TRIALS):
            x_sample, y_sample = classSampling(x_train, y_train, spc, NUM_CLASSES_TURNEDTABLE)

            svm = SVC(C=1.0, decision_function_shape = 'ovo', kernel="linear")
            svm.fit(x_sample, y_sample)

            train_acc = svm.score(x_sample, y_sample)
            test_acc = svm.score(x_test, y_test)

            print("SPC {} Train Accuracy: {:.3f}".format(spc, train_acc))
            print("SPC {} Test Accuracy: {:.3f}".format(spc, test_acc))
            print()

            accuracies.append(test_acc)

        mean_acc = np.mean(accuracies)
        std_acc = np.std(accuracies)

        mean_acc = round(100 * mean_acc, 3)
        std_acc = round(100 * std_acc, 3)

        print("After {} trials - Test Accuracy is {} +- {}".format(TRIALS, mean_acc, std_acc ))
        print("------------------------------------------------------------------------------")
        print()

In [ ]:
train_svm_with_spc(x_train_batch_norm_48, y_train, x_test_batch_norm_48, y_test)

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.594

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.586

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.580

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.552

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.605

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.600

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.572

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.597

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.577

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.560

After 10 trials - Test Accuracy is 58.217 +- 1.657
------------------------------------------------------------------------------

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.679

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.653

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.678

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.653

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.651


In [20]:
train_svm_with_spc(x_train_batch_norm_49, y_train, x_test_batch_norm_49, y_test)


SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.620

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.681

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.713

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.691

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.688

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.690

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.662

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.699

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.712

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.729

After 10 trials - Test Accuracy is 68.853 +- 2.884
------------------------------------------------------------------------------

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.812

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.795

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.795

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.805

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.775


In [21]:
train_svm_with_spc(x_train_batch_norm_50, y_train, x_test_batch_norm_50, y_test)


SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.648

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.684

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.628

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.670

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.647

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.625

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.662

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.679

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.640

SPC 10 Train Accuracy: 1.000
SPC 10 Test Accuracy: 0.569

After 10 trials - Test Accuracy is 64.512 +- 3.174
------------------------------------------------------------------------------

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.722

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.738

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.744

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.707

SPC 20 Train Accuracy: 1.000
SPC 20 Test Accuracy: 0.718
